# 2d video to 3D

This code makes .blend file from single video

 ### 2d Pose Estimation using Openpose
 - make images from video
 - 2d pose estimation by openpose
 - target video should be at '/openpose/sample_videos'
 - images create at  '/openpose/sample_images'
 - after 2d pose estimation json files create at '/openpose/sample_jsons'

In [ ]:
import os
os.chdir('openpose')
!bash video_to_images.sh 24
!bash 2d_estimation.sh
os.chdir('..')

### Set Ground Position for Estimation
- click 'left top','left bottom','right bottom','right bottom' in order
- writes 'points' file at directory

In [ ]:
%%html
<style>
.output_wrapper button.btn.btn-default,
.output_wrapper .ui-dialog-titlebar {
  display: none;
}
</style>

In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt
import matplotlib as mpl 
import numpy as np
import matplotlib.lines as mlines
import ipywidgets as wdg
from IPython.display import display

points = []
prev = []
cnt = 0

a = plt.imread("sample.png")
fig = plt.figure()
plt.imshow(a)

def write():
    global points
    f = open("points",'w+')
    tmp = [str(x)+" "+str(y) for (x,y) in points]
    f.write("\n".join(tmp))
    f.close()

def onclick(event):
    global cnt,points,prev,msg
    if cnt == 4:
        return
    x = float(event.xdata)
    y = float(event.ydata)
    points.append([x,y])
    cnt += 1
    plt.scatter(x,y,s=10,c='white')
    if cnt>1:
            now = [x,y]
            line = plt.Polygon([prev,now], closed=None, fill=None, edgecolor='w')
            plt.gca().add_line(line)
            if cnt == 4:
                line = plt.Polygon([now,points[0]], closed=None, fill=None, edgecolor='w')
                area = plt.Polygon(points,closed=True,fill=True)
                plt.gca().add_line(line)
                plt.gca().add_line(area)
                write()
    prev = [x,y]
    
ka = fig.canvas.mpl_connect('button_press_event',onclick)


### Camera Calibration
- creates ground position information
- you need to enter ratio of the ground (default value: width=3 height=2)

In [ ]:
# width height ratio
! python calibration.py 3 2

### 3d Pose Estimation Using HMR
- you need to enter number of people to estimate (default value: 1)
- after 3d pose estimation it creates csv files at '/hmr/output/csv', 'hmr/output/csv_joined'

In [ ]:
! bash hmr/mong_3dpose_estimate.sh 1

### Make .bvh file Using Blender
- you need to enter number of people to estimate (default value: 1)
- you need to enter the name of bvh file
- output files at 'hmr/output/bvh_animation/'

In [ ]:
# -- [number of people] [output filename]
blender --background hmr/csv_to_bvh.blend -noaudio -P hmr/mong_csv_to_bvh.py -- 1 squash1


### Make .blend file using Blender and Makehuman Model
- you need to enter bvh filename, frame number, number of people 
- output .blend files at current directory

In [ ]:
# -- [file_name] [frame_num] [people_num]
blender --background --enable-autoexec --python render.py squash1 100 1